Created on Fri Jan 28 11:34:21 2019 <br>
@author: zoldi.miklos
# <h1><center> Changing filter cubes on the old and new STORM setup </center></h1>

Dual-color STORM issue: imaging sequentially with 647 and then 568 laser would be promising, as there is no channel crosstalk and one can detect the signal specifically from that fluorophore for which the appropriate filter cube was chosen. <br>
***
However, as these setups are not high-precision ones, it is not ensured that during filter cube change, the second one would have the very same position and angle as the first one. Hence, the distance between far-red and red fluorophores would not only depend on the chromatic aberration, but on the magnitude of the above effect. <br>
- If this effect is big enough, confocal images can not be discarded, and aligning can only be done onto the respective confocal images. The spatial resolution of the two targtes respective to each other would be limited by the confocal microscope's resolution and by our ability to align the two different image modalities to each other.
- If the effect of filter cube change is neglible, one only has to handle the magnitude of warp correction (which could be calculated).

### Procedure: 
- 100 nm Tetraspeck beads dissolved in DPBS and coated on coverslip
- Imaging them with one color using 'A' filter cube, then changing to 'B' filter cube, and then doing this mulitple times (s001-s005)
- Imagings were taken at several positions (position 1-3)
- As only 1 laser line is used, the displacement between clusters should correspond only to the effect of filter cube change, as there is no chromatic aberration as in the case of 2 laser lines. 
- The drift was calculated manually (software could not do it on beads), and was small enough not to handle its effect between imagings.

### Notebook
- Circles mark the positon of clusters - Zoom into one cluster (enable tools on the top of the image), then click on the legends to show the clusters of the given storm image. 
- Hover on the circles (middle of cluters) to see their xy coordinates

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
#import modules
import os
import numpy as np
import pandas as pd
import itertools
from sklearn.cluster import DBSCAN

In [1]:
#import and set plotting styles
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
from bokeh.models import ColumnDataSource, HoverTool, Legend, Arrow, NormalHead
from bokeh.models.annotations import Label
from bokeh.palettes import Blues, Oranges
output_notebook(hide_banner=True)

In [5]:
#get files
basepath = 'L:/Miki/STORM_projects/Filter_cube_changes/190124_new_old_system//' # location of folders
storm1_488_path = basepath + 'STORM1_quad_dual_488//'
storm1_647_path = basepath + 'STORM1_storm_dual_647//'
storm2_488_path = basepath + 'STORM2_quad_fitc_488//'

storm1_488_files = [i for i in os.listdir(storm1_488_path) if i.endswith('bin.txt')]
storm1_647_files = [i for i in os.listdir(storm1_647_path) if i.endswith('bin.txt')]
storm2_488_files = [i for i in os.listdir(storm2_488_path) if i.endswith('bin.txt')]

In [11]:
def plot_beads (path = storm1_488_path, files = storm1_488_files, cube1 = 'QUAD', cube2 = 'DUAL', posNr=1,
               density_filter = dict(min_samples=3, eps=200, metric='euclidean', n_jobs=-1)):
    
    #configuring hover tool being active just on the figure named "center"
    hover = HoverTool(names=["center"])
    TOOLTIPS = [("(x,y)", "(@x{int}, @y{int})")] #get xy values of the dataspace as integers 
    
    #initialize figure and lists
    p1 = figure(match_aspect=True, x_axis_label='x-coordinates', y_axis_label='y-coordinates', 
                height=500, width=950, tools=["pan","wheel_zoom","box_zoom","reset", hover], tooltips=TOOLTIPS)
    legend1, legend2 = [], [] 
    plot1, plot2 = [], []
    drift1, drift2 = [], []

    for fname, color1, color2 in zip (files, itertools.cycle(Blues[5]),itertools.cycle(Oranges[5])):
        items = fname.split('_')
        setup, pos, cube, laser, count = items[0], items[3], items[4], items[5], items[6]
        #plot results from only 1 position
        if pos != pos[:-1] + str(posNr):
            continue
        #read file as pd dataframe, get xy column as numpy array 
        dt = pd.read_csv(path+fname, delimiter='\t')        
        xy_col = dt[['X','Y']].values
        xyc_col = dt[['Xc','Yc']].values #drift corrected xy coord
        #check amount of drift correction 
        xy_drift = xyc_col[-1] - xy_col[-1] #last column (where drift should be maximal)
        
        #apply density filter on xyc (just on drift corrected ones, you should only use them later)
        dbscan = DBSCAN(**density_filter)
        labels = dbscan.fit_predict(xyc_col)
        xyc_col = xyc_col[labels != -1] #-1: noise
        labels = labels[labels != -1] #also excluding from clusters
        
        #search for center of clusters to plot and hover over it
        clusterID = np.unique(labels)
        center_array = np.array([[]])
        for i in clusterID:        
            cluster_i = xyc_col[labels == i]
            cluster_i_center = np.mean(cluster_i, axis=0)      
            cluster_i_center = cluster_i_center[np.newaxis, :]
            if not center_array.any(): #first cluster        
                center_array = cluster_i_center        
            else:  #array to append the cluster's center
                center_array = np.concatenate((center_array, cluster_i_center), axis=0)
        
        #calculates amount of drift if no drift correction was applied 
        #mean_drift = average of all clusters' (result of DBSCAN clustering) drift 
        if int(xy_drift[0]) == 0 and int(xy_drift[1]) == 0:
            clusterID = np.unique(labels)
            mean_drift = np.array([0.0, 0.0])
            #iterate over all clusters
            index = 0
            for i in clusterID:          
                cluster_i = xyc_col[labels == i]
                quarter = len(cluster_i)/4
                if quarter < 10: #not even 40 points in total in a cluster; mean would be calculated from
                                 #less than 10 points; if less than 4: x/4 = 0, NaN is created and script crashes 
                    np.delete(clusterID, index)
                    continue
                q1 = cluster_i[:quarter, :]
                q4 = cluster_i[3*quarter:, :]
                mean_q1, mean_q4 = np.mean(q1, axis=0), np.mean(q4, axis=0)
                drift = mean_q4 - mean_q1
                mean_drift += drift
                index += 1
            mean_drift = mean_drift / len(clusterID)

        #plot figure1
        if cube == cube1:
            glyph = p1.scatter(xyc_col[:,0], xyc_col[:,1], marker='circle', color=color1)
            legend1.append(count + "_" + cube)
            plot1.append(glyph)
            drift1.append(mean_drift)

        if cube == cube2:
            glyph = p1.scatter(xyc_col[:,0], xyc_col[:,1], marker='circle', color=color2)
            legend2.append(count + "_" + cube)
            plot2.append(glyph)
            drift2.append(mean_drift)
        glyph.visible = False

        #plot figure1 centers to hover on
        center_glyph = p1.circle(center_array[:,0], center_array[:,1], fill_color='white', fill_alpha=0.2,
                             line_color='black', line_alpha=0.5, line_width=2, size=15, name="center")
    #handle legends and title  
    legendSet1 = Legend(items=[(i, [j]) for i,j in zip (legend1, plot1)], location= (600, 170)) # ~-284 for 5 legends, 170 for 10
    p1.add_layout(legendSet1)
    legendSet2 = Legend(items=[(i, [j]) for i,j in zip (legend2, plot2)], location= 'top_right') 
    p1.add_layout(legendSet2)
    p1.legend.click_policy = 'hide'
    p1.title.text = "%s - imaging 100 nm Tetraspeck beads, %s/%s filter cube changes - position: %d" % (setup, cube1, cube2, posNr)

    #plot figure2
    title2 = "drifts during imaging (%s)" % cube1 
    p2 = figure(title=title2, match_aspect=True, x_axis_label='x-drift (nm)', y_axis_label='y-drift (nm)', 
                height=400, width=400)
    for drift, legend, color in zip (drift1, legend1, itertools.cycle(Blues[5])):
        p2.circle(x=[0, drift[0]], y=[0, drift[1]], color=color)
        p2.add_layout(Arrow(end=NormalHead(fill_color=color), x_start=0,
                            y_start=0, x_end=drift[0], y_end=drift[1]))
        p2.add_layout(Label(text=legend.split('_')[0], x=drift[0],
                            y=drift[1], x_offset=5, y_offset=5))
    #plot figure3
    title3 = "drifts during imaging (%s)" % cube2
    p3 = figure(title=title3, match_aspect=True, x_axis_label='x-drift (nm)', y_axis_label='y-drift (nm)', 
                height=400, width=400)
    for drift, legend, color in zip (drift2, legend2, itertools.cycle(Oranges[5])):
        p3.circle(x=[0, drift[0]], y=[0, drift[1]], color=color)
        p3.add_layout(Arrow(end=NormalHead(fill_color=color), x_start=0,
                            y_start=0, x_end=drift[0], y_end=drift[1]))
        p3.add_layout(Label(text=legend.split('_')[0], x=drift[0],
                            y=drift[1], x_offset=5, y_offset=5))
    #manage and show gridplots
    grid = gridplot([[p1], [p2, p3]])
    show(grid)

## _STORM1_ - changing <font color='red'> DUAL and QUAD </font> cubes, imaging with 488 laser

In [9]:
for i in [1]:
    plot_beads(path = storm1_488_path, files = storm1_488_files, cube1 = 'DUAL', cube2 = 'QUAD', posNr=i,
               density_filter = dict(min_samples=3, eps=200, metric='euclidean', n_jobs=-1))

## _STORM1_ - changing <font color='red'> STORM and DUAL </font> cubes, imaging with 647 laser

In [10]:
for i in [1]:
    plot_beads(path = storm1_647_path, files = storm1_647_files, cube1 = 'STORM', cube2 = 'DUAL', posNr=i,
               density_filter = dict(min_samples=3, eps=200, metric='euclidean', n_jobs=-1))

## _STORM2_ - changing <font color='red'> FITC and QUAD </font> cubes, imaging with 488 laser

In [12]:
for i in [1]:
    plot_beads(path =storm2_488_path, files = storm2_488_files, cube1 = 'FITC', cube2 = 'QUAD', posNr=i,
               density_filter = dict(min_samples=3, eps=200, metric='euclidean', n_jobs=-1))


# Conclusion: 

- New setup is not "more precise" than the old one
- Precision may depend on the filter cubes and their location - which seems that can be maintained when changing filter cubes -> can it be determined and could we use this value to correct for data on biological samples?
- Should check how far-red / red filter cube changes look like (could not do it as beads bleached quickly when imagign with 561)

***
## To-dos <br>

- calculate amount of shift between corresponding channels over changes (mean quad1 vs mean fitc1; mean quad2 vs mean fitc2).
    - Using Bálint's align code?
    - Using smaller Min Height not to throw out clusters and to have identical cluster numbers?
- do it on biological fata, farred/red imagings: my bassoon and Susanne's drug data   